In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [68]:
from google.colab import files
uploaded = files.upload()

Saving SalaryData_Train.csv to SalaryData_Train.csv


salary train

In [70]:
salary = pd.read_csv("SalaryData_Train.csv")
salary

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30156,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
30157,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
30158,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
30159,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [71]:
salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   educationno    30161 non-null  int64 
 4   maritalstatus  30161 non-null  object
 5   occupation     30161 non-null  object
 6   relationship   30161 non-null  object
 7   race           30161 non-null  object
 8   sex            30161 non-null  object
 9   capitalgain    30161 non-null  int64 
 10  capitalloss    30161 non-null  int64 
 11  hoursperweek   30161 non-null  int64 
 12  native         30161 non-null  object
 13  Salary         30161 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


In [72]:
salary.describe()

,age,educationno,capitalgain,capitalloss,hoursperweek
count,30161.000000,30161.000000,30161.000000,30161.000000,30161.000000
mean,38.438115,10.121316,1092.044064,88.302311,40.931269
std,13.134830,2.550037,7406.466611,404.121321,11.980182
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,47.000000,13.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


In [74]:
salary.duplicated().sum()

3258

In [75]:
salary1 = salary.drop_duplicates()

In [76]:
salary1

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30154,53,Private,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
30155,22,Private,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
30156,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
30158,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


In [77]:
salary1.corr()

,age,educationno,capitalgain,capitalloss,hoursperweek
age,1.000000,0.038048,0.077235,0.052684,0.072529
educationno,0.038048,1.000000,0.126085,0.080025,0.145491
capitalgain,0.077235,0.126085,1.000000,-0.036023,0.079403
capitalloss,0.052684,0.080025,-0.036023,1.000000,0.049093
hoursperweek,0.072529,0.145491,0.079403,0.049093,1.000000


In [78]:
label_encoder=preprocessing.LabelEncoder()
salary1['maritalstatus']=label_encoder.fit_transform(salary1['maritalstatus'])
salary1['age']=label_encoder.fit_transform(salary1['age'])
salary1['workclass']=label_encoder.fit_transform(salary1['workclass'])
salary1['education']=label_encoder.fit_transform(salary1['education'])
salary1['occupation']=label_encoder.fit_transform(salary1['occupation'])
salary1['relationship']=label_encoder.fit_transform(salary1['relationship'])
salary1['race']=label_encoder.fit_transform(salary1['race'])
salary1['sex']=label_encoder.fit_transform(salary1['sex'])
salary1['native']=label_encoder.fit_transform(salary1['native'])

<ipython-input-78-4e56f2866ceb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary1['maritalstatus']=label_encoder.fit_transform(salary1['maritalstatus'])
<ipython-input-78-4e56f2866ceb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary1['age']=label_encoder.fit_transform(salary1['age'])
<ipython-input-78-4e56f2866ceb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [79]:
salary1

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,22,5,9,13,4,0,1,4,1,2174,0,40,37,<=50K
1,33,4,9,13,2,3,0,4,1,0,0,13,37,<=50K
2,21,2,11,9,0,5,1,4,1,0,0,40,37,<=50K
3,36,2,1,7,2,5,0,2,1,0,0,40,37,<=50K
4,11,2,9,13,2,9,5,2,0,0,0,40,4,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30154,36,2,12,14,2,3,0,4,1,0,0,40,37,>50K
30155,5,2,15,10,4,10,1,4,1,0,0,40,37,<=50K
30156,10,2,7,12,2,12,5,4,0,0,0,38,37,<=50K
30158,41,2,11,9,6,0,4,4,0,0,0,40,37,<=50K


salary test

In [81]:
from google.colab import files
uploaded = files.upload()

Saving SalaryData_Test.csv to SalaryData_Test (3).csv


In [80]:
salarytest = pd.read_csv("SalaryData_Test.csv")
salarytest

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15055,33,Private,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K
15056,39,Private,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
15057,38,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
15058,44,Private,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [82]:
salarytest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15060 entries, 0 to 15059
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            15060 non-null  int64 
 1   workclass      15060 non-null  object
 2   education      15060 non-null  object
 3   educationno    15060 non-null  int64 
 4   maritalstatus  15060 non-null  object
 5   occupation     15060 non-null  object
 6   relationship   15060 non-null  object
 7   race           15060 non-null  object
 8   sex            15060 non-null  object
 9   capitalgain    15060 non-null  int64 
 10  capitalloss    15060 non-null  int64 
 11  hoursperweek   15060 non-null  int64 
 12  native         15060 non-null  object
 13  Salary         15060 non-null  object
dtypes: int64(5), object(9)
memory usage: 1.6+ MB


In [83]:
salarytest.describe()

,age,educationno,capitalgain,capitalloss,hoursperweek
count,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000
mean,38.768327,10.112749,1120.301594,89.041899,40.951594
std,13.380676,2.558727,7703.181842,406.283245,12.062831
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,13.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,3770.000000,99.000000


In [84]:
salarytest.duplicated().sum()

930

In [85]:
salary_T = salarytest.drop_duplicates()

In [86]:
salary_T

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15055,33,Private,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K
15056,39,Private,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
15057,38,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
15058,44,Private,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [87]:
label_encoder=preprocessing.LabelEncoder()
salary_T['maritalstatus']=label_encoder.fit_transform(salary_T['maritalstatus'])
salary_T['age']=label_encoder.fit_transform(salary_T['age'])
salary_T['workclass']=label_encoder.fit_transform(salary_T['workclass'])
salary_T['education']=label_encoder.fit_transform(salary_T['education'])
salary_T['occupation']=label_encoder.fit_transform(salary_T['occupation'])
salary_T['relationship']=label_encoder.fit_transform(salary_T['relationship'])
salary_T['race']=label_encoder.fit_transform(salary_T['race'])
salary_T['sex']=label_encoder.fit_transform(salary_T['sex'])
salary_T['native']=label_encoder.fit_transform(salary_T['native'])

<ipython-input-87-881640a12db9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary_T['maritalstatus']=label_encoder.fit_transform(salary_T['maritalstatus'])
<ipython-input-87-881640a12db9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary_T['age']=label_encoder.fit_transform(salary_T['age'])
<ipython-input-87-881640a12db9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [88]:
salary_T

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,8,2,1,7,4,6,3,2,1,0,0,40,37,<=50K
1,21,2,11,9,2,4,0,4,1,0,0,50,37,<=50K
2,11,1,7,12,2,10,0,4,1,0,0,40,37,>50K
3,27,2,15,10,2,6,0,2,1,7688,0,40,37,>50K
4,17,2,0,6,4,7,1,4,1,0,0,30,37,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15055,16,2,9,13,4,9,3,4,1,0,0,40,37,<=50K
15056,22,2,9,13,0,9,1,4,0,0,0,36,37,<=50K
15057,21,2,9,13,2,9,0,4,1,0,0,50,37,<=50K
15058,27,2,9,13,0,0,3,1,1,5455,0,40,37,<=50K


In [89]:
x_train=salary1.iloc[:,0:13]
y_train=salary1['Salary']
x_test=salary_T.iloc[:,0:13]
y_test=salary_T['Salary']

In [90]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((26903, 13), (14130, 13), (26903,), (14130,))

In [91]:
x_train

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native
0,22,5,9,13,4,0,1,4,1,2174,0,40,37
1,33,4,9,13,2,3,0,4,1,0,0,13,37
2,21,2,11,9,0,5,1,4,1,0,0,40,37
3,36,2,1,7,2,5,0,2,1,0,0,40,37
4,11,2,9,13,2,9,5,2,0,0,0,40,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30154,36,2,12,14,2,3,0,4,1,0,0,40,37
30155,5,2,15,10,4,10,1,4,1,0,0,40,37
30156,10,2,7,12,2,12,5,4,0,0,0,38,37
30158,41,2,11,9,6,0,4,4,0,0,0,40,37


In [92]:
x_test

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native
0,8,2,1,7,4,6,3,2,1,0,0,40,37
1,21,2,11,9,2,4,0,4,1,0,0,50,37
2,11,1,7,12,2,10,0,4,1,0,0,40,37
3,27,2,15,10,2,6,0,2,1,7688,0,40,37
4,17,2,0,6,4,7,1,4,1,0,0,30,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15055,16,2,9,13,4,9,3,4,1,0,0,40,37
15056,22,2,9,13,0,9,1,4,0,0,0,36,37
15057,21,2,9,13,2,9,0,4,1,0,0,50,37
15058,27,2,9,13,0,0,3,1,1,5455,0,40,37


In [93]:
y_train

0         <=50K
1         <=50K
2         <=50K
3         <=50K
4         <=50K
          ...  
30154      >50K
30155     <=50K
30156     <=50K
30158     <=50K
30160      >50K
Name: Salary, Length: 26903, dtype: object

In [94]:
y_test

0         <=50K
1         <=50K
2          >50K
3          >50K
4         <=50K
          ...  
15055     <=50K
15056     <=50K
15057     <=50K
15058     <=50K
15059      >50K
Name: Salary, Length: 14130, dtype: object

Naive Bayes - Multinomial

In [95]:
from sklearn.naive_bayes import MultinomialNB as MB
# Multinomial Naive Bayes

# Model Train
classifier_mb = MB()

classifier_mb.fit(x_train, y_train)

# Model Test

# Model Accuracy on train set
train_pred_m = classifier_mb.predict(x_train)
accuracy_train_m = np.mean(train_pred_m==y_train) 

# Model Accuracy on test set
test_pred_m = classifier_mb.predict(x_test)
accuracy_test_m = np.mean(test_pred_m==y_test) 

In [96]:
accuracy_train_m

0.7683158012117608

In [97]:
test_pred_m

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' >50K', ' <=50K'],
      dtype='<U6')

In [98]:
accuracy_test_m

0.7733899504600141

Naive Bayes - Gaussian

In [99]:
from sklearn.naive_bayes import GaussianNB as GB
# Multinomial Naive Bayes

# Model Train
classifier_mb = GB()

classifier_mb.fit(x_train, y_train)

# Model Test

# Model Accuracy on train set
train_pred_g = classifier_mb.predict(x_train)
accuracy_train_g = np.mean(train_pred_m==y_train) 

# Model Accuracy on test set
test_pred_g = classifier_mb.predict(x_test)
accuracy_test_g = np.mean(test_pred_m==y_test) 

In [100]:
accuracy_train_g

0.7683158012117608

In [101]:
test_pred_g

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' >50K', ' <=50K'],
      dtype='<U6')

In [102]:
accuracy_test_g

0.7733899504600141

Naive Bayes - Bernoulli

In [103]:
from sklearn.naive_bayes import BernoulliNB as nb
# Multinomial Naive Bayes

# Model Train
classifier_mb = nb()

classifier_mb.fit(x_train, y_train)

# Model Test

# Model Accuracy on train set
train_pred_n = classifier_mb.predict(x_train)
accuracy_train_n= np.mean(train_pred_m==y_train) 

# Model Accuracy on test set
test_pred_n = classifier_mb.predict(x_test)
accuracy_test_n = np.mean(test_pred_m==y_test) 

In [104]:
accuracy_train_n

0.7683158012117608

In [105]:
test_pred_n

array([' <=50K', ' >50K', ' >50K', ..., ' >50K', ' <=50K', ' >50K'],
      dtype='<U6')

In [106]:
accuracy_test_n

0.7733899504600141